<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1>

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1.  <a href="#item1">Download and Explore Dataset</a>

2.  <a href="#item2">Explore Neighborhoods in Toronto</a>

3.  <a href="#item3">Analyze Each Neighborhood</a>

4.  <a href="#item4">Cluster Neighborhoods</a>

5.  <a href="#item5">Examine Clusters</a>  
    </font>
    </div>


Before we get the data and start exploring it, let's download all the dependencies that we will need.


In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


<a id='item1'></a>


## 1. Download and Explore Dataset


Neighborhood has a total of 5 boroughs and 306 neighborhoods. In order to segement the neighborhoods and explore them, we will essentially need a dataset that contains the 5 boroughs and the neighborhoods that exist in each borough as well as the the latitude and logitude coordinates of each neighborhood. 


For your convenience, I downloaded the files and placed it on the server, so you can simply run a `wget` command and access the data. So let's go ahead and do that.


In [7]:
import requests # library to handle requests
url = 'https://raw.githubusercontent.com/adamw523/toronto-geojson/master/neighbourhoods.js'
url
#data = requests.get(url).json()

'https://raw.githubusercontent.com/adamw523/toronto-geojson/master/neighbourhoods.js'

#### Load and explore the data


Next, let's load the data.


In [10]:
with open('neighbourhoods.json') as json_data:
    toronto_data = json.load(json_data)

Let's take a quick look at the data.


In [11]:
toronto_data

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'DAUID': '35200879',
    'PRUID': '35',
    'CSDUID': '3520005',
    'HOODNUM': 81,
    'HOOD': 'Trinity-Bellwoods',
    'FULLHOOD': 'Trinity-Bellwoods (81)'},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-79.40428280044927, 43.64797961606815],
      [-79.403956753622, 43.64718271074494],
      [-79.42236786578222, 43.643467621011894],
      [-79.42640543946513, 43.65360764326518],
      [-79.41868792113178, 43.65521730993704],
      [-79.41769878521191, 43.65524323486715],
      [-79.41514736685951, 43.65496322517198],
      [-79.40767889826175, 43.65646442447146],
      [-79.40428280044927, 43.64797961606815]]]}},
  {'type': 'Feature',
   'properties': {'DAUID': '35201763',
    'PRUID': '35',
    'CSDUID': '3520005',
    'HOODNUM': 1,
    'HOOD': 'West Humber-Clairville',
    'FULLHOOD': 'West Humber-Clairville (1)'},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-79.5666757498039

Notice how all the relevant data is in the _features_ key, which is basically a list of the neighborhoods. So, let's define a new variable that includes this data.


In [12]:
neighborhoods_data = toronto_data['features']

Let's take a look at the first item in this list.


In [13]:
neighborhoods_data[0]

{'type': 'Feature',
 'properties': {'DAUID': '35200879',
  'PRUID': '35',
  'CSDUID': '3520005',
  'HOODNUM': 81,
  'HOOD': 'Trinity-Bellwoods',
  'FULLHOOD': 'Trinity-Bellwoods (81)'},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-79.40428280044927, 43.64797961606815],
    [-79.403956753622, 43.64718271074494],
    [-79.42236786578222, 43.643467621011894],
    [-79.42640543946513, 43.65360764326518],
    [-79.41868792113178, 43.65521730993704],
    [-79.41769878521191, 43.65524323486715],
    [-79.41514736685951, 43.65496322517198],
    [-79.40767889826175, 43.65646442447146],
    [-79.40428280044927, 43.64797961606815]]]}}

In [30]:
neighborhoods_data[0]['geometry']['coordinates'][0][0]

[-79.40428280044927, 43.64797961606815]

#### Tranform the data into a _pandas_ dataframe


The next task is essentially transforming this data of nested Python dictionaries into a _pandas_ dataframe. So let's start by creating an empty dataframe.


In [34]:
# define the dataframe columns
# Borough: HOOD
# Neighborhood : FULLHOOD
# Latitude = neighborhoods_data[0]['geometry']['coordinates'][0][0][0]
# Longitude = neighborhoods_data[0]['geometry']['coordinates'][0][0][1]
column_names = ['Hood', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

Take a look at the empty dataframe to confirm that the columns are as intended.


In [35]:
neighborhoods

,Hood,Neighborhood,Latitude,Longitude


Then let's loop through the data and fill the dataframe one row at a time.


In [44]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['HOOD'] 
    neighborhood_name = data['properties']['FULLHOOD']
        
    neighborhood_latlon = data['geometry']['coordinates'][0][0]
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Hood': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

Quickly examine the resulting dataframe.


In [45]:
# 纬度经度 Latitude	Longitude  Borough 区
neighborhoods.head()

,Hood,Neighborhood,Latitude,Longitude
0,Trinity-Bellwoods,Trinity-Bellwoods (81),43.64798,-79.404283
1,West Humber-Clairville,West Humber-Clairville (1),43.711786,-79.566676
2,Mount Olive-Silverstone-Jamestown,Mount Olive-Silverstone-Jamestown (2),43.735516,-79.578253
3,Humber Summit,Humber Summit (21),43.748808,-79.557618
4,Thistletown-Beaumond Heights,Thistletown-Beaumond Heights (3),43.729598,-79.5539


And make sure that the dataset has all 5 boroughs and 306 neighborhoods.


In [47]:
# .unique()
print('The dataframe has {} Hoods and {} neighborhoods.'.format(
        len(neighborhoods['Hood'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 140 Hoods and 280 neighborhoods.


#### Use geopy library to get the latitude and longitude values of New York City.


In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>ny_explorer</em>, as shown below.


In [48]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.6534817, -79.3839347.


#### Create a map of Toronto with neighborhoods superimposed on top.


In [50]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Hood'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

**Folium** is a great visualization library. Feel free to zoom into the above map, and click on each circle mark to reveal the name of the neighborhood and its respective borough.


However, for illustration purposes, let's simplify the above map and segment and cluster only the neighborhoods in Manhattan. So let's slice the original dataframe and create a new dataframe of the Manhattan data.


In [57]:
humber_summit_data = neighborhoods[neighborhoods['Hood'] == 'Humber Summit'].reset_index(drop=True)
humber_summit_data.head()

,Hood,Neighborhood,Latitude,Longitude
0,Humber Summit,Humber Summit (21),43.748808,-79.557618
1,Humber Summit,Humber Summit (21),43.748808,-79.557618


Let's get the geographical coordinates of Manhattan.


In [58]:
address = 'Humber Summit'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Humber Summit are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Humber Summit are 43.7600778, -79.5717598.


As we did with all of New York City, let's visualizat Manhattan the neighborhoods in it.


In [59]:
# create map of Humber_Summit using latitude and longitude values
map_humber_summit = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(humber_summit_data['Latitude'], humber_summit_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_humber_summit)  
    
map_humber_summit

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.


#### Define Foursquare Credentials and Version


In [60]:
CLIENT_ID = 'CPVIOYTO132HU0BARYUBBAYTYJUHDMIXQ3UQMD1G0QITTGEW' # your Foursquare ID
CLIENT_SECRET = '01WA1LVGQ2TY0ACDY0WBWOI0YXKY1CZTOEB1DI40FPCFOVNF' # your Foursquare Secret

VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: CPVIOYTO132HU0BARYUBBAYTYJUHDMIXQ3UQMD1G0QITTGEW
CLIENT_SECRET:01WA1LVGQ2TY0ACDY0WBWOI0YXKY1CZTOEB1DI40FPCFOVNF


#### Let's explore the first neighborhood in our dataframe.


Get the neighborhood's name.


In [61]:
humber_summit_data.loc[0, 'Neighborhood']

'Humber Summit (21)'

Get the neighborhood's latitude and longitude values.


In [68]:
neighborhood_latitude = humber_summit_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = humber_summit_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = humber_summit_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Humber Summit (21) are 43.74880789315456, -79.55761827656043.


#### Now, let's get the top 100 venues that are in Marble Hill within a radius of 500 meters.


First, let's create the GET request URL. Name your URL **url**.


In [69]:
# type your answer here
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=CPVIOYTO132HU0BARYUBBAYTYJUHDMIXQ3UQMD1G0QITTGEW&client_secret=01WA1LVGQ2TY0ACDY0WBWOI0YXKY1CZTOEB1DI40FPCFOVNF&v=20180605&ll=43.74880789315456,-79.55761827656043&radius=500&limit=100'

Double-click **here** for the solution.

<!-- The correct answer is:
LIMIT = 100 # limit of number of venues returned by Foursquare API
-->

<!--
radius = 500 # define radius
-->

<!--
\\\\ # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL
--> 


Send the GET request and examine the resutls


In [70]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '6022053f755d611ffc1be606'},
 'response': {'headerLocation': 'Humber Summit',
  'headerFullLocation': 'Humber Summit, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 6,
  'suggestedBounds': {'ne': {'lat': 43.753307897654565,
    'lng': -79.55140047881106},
   'sw': {'lat': 43.74430788865456, 'lng': -79.5638360743098}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4cab739da6e08cfa1fd3a594',
       'name': 'Hakka No.1',
       'location': {'address': '2440 Finch Ave W',
        'lat': 43.750245811612785,
        'lng': -79.55202705615771,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.750245811612785,
          'lng': -79.55202705615771}],
        'distance': 477,
        'postalCode': '

From the Foursquare lab in the previous module, we know that all the information is in the _items_ key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.


In [71]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a _pandas_ dataframe.


In [72]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-72-561c05f0fdd1>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Hakka No.1,Asian Restaurant,43.750246,-79.552027
1,Shoppers Drug Mart,Pharmacy,43.748207,-79.561000
2,TD Canada Trust Branch and ATM,Bank,43.748880,-79.561191
3,Finchdale Plaza,Shopping Mall,43.748722,-79.560473
4,Albertos Trattoria,Italian Restaurant,43.748744,-79.560714


And how many venues were returned by Foursquare?


In [73]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

6 venues were returned by Foursquare.


<a id='item2'></a>


## 2. Explore Neighborhoods in Manhattan


#### Let's create a function to repeat the same process to all the neighborhoods in Manhattan


In [74]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called _manhattan_venues_.


In [75]:
# type your answer here
humber_summit_venues = getNearbyVenues(names=humber_summit_data['Neighborhood'],
                                   latitudes=humber_summit_data['Latitude'],
                                   longitudes=humber_summit_data['Longitude']
                                  )

Humber Summit (21)
Humber Summit (21)


Double-click **here** for the solution.

<!-- The correct answer is:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )
--> 


#### Let's check the size of the resulting dataframe


In [76]:
print(humber_summit_venues.shape)
humber_summit_venues.head()

(12, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Humber Summit (21),43.748808,-79.557618,Hakka No.1,43.750246,-79.552027,Asian Restaurant
1,Humber Summit (21),43.748808,-79.557618,Shoppers Drug Mart,43.748207,-79.561000,Pharmacy
2,Humber Summit (21),43.748808,-79.557618,TD Canada Trust Branch and ATM,43.748880,-79.561191,Bank
3,Humber Summit (21),43.748808,-79.557618,Finchdale Plaza,43.748722,-79.560473,Shopping Mall
4,Humber Summit (21),43.748808,-79.557618,Albertos Trattoria,43.748744,-79.560714,Italian Restaurant


Let's check how many venues were returned for each neighborhood


In [77]:
humber_summit_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Humber Summit (21),12,12,12,12,12,12


#### Let's find out how many unique categories can be curated from all the returned venues


In [78]:
print('There are {} uniques categories.'.format(len(humber_summit_venues['Venue Category'].unique())))

There are 6 uniques categories.


<a id='item3'></a>


## 3. Analyze Each Neighborhood


In [105]:
# one hot encoding
humber_summit_onehot = pd.get_dummies(humber_summit_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
humber_summit_onehot['Neighborhood'] = humber_summit_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [humber_summit_onehot.columns[-1]] + list(humber_summit_onehot.columns[:-1])
humber_summit_onehot = humber_summit_onehot[fixed_columns]

humber_summit_onehot.head()

,Neighborhood,Asian Restaurant,Bank,Electronics Store,Italian Restaurant,Pharmacy,Shopping Mall
0,Humber Summit (21),1,0,0,0,0,0
1,Humber Summit (21),0,0,0,0,1,0
2,Humber Summit (21),0,1,0,0,0,0
3,Humber Summit (21),0,0,0,0,0,1
4,Humber Summit (21),0,0,0,1,0,0


And let's examine the new dataframe size.


In [106]:
humber_summit_onehot.shape

(12, 7)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category


In [107]:
humber_summit_grouped = humber_summit_onehot.groupby('Neighborhood').mean().reset_index()
humber_summit_grouped

,Neighborhood,Asian Restaurant,Bank,Electronics Store,Italian Restaurant,Pharmacy,Shopping Mall
0,Humber Summit (21),0.166667,0.166667,0.166667,0.166667,0.166667,0.166667


#### Let's confirm the new size


In [108]:
humber_summit_grouped.shape

(1, 7)

#### Let's print each neighborhood along with the top 5 most common venues


In [109]:
num_top_venues = 5

for hood in humber_summit_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = humber_summit_grouped[humber_summit_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Humber Summit (21)----
                venue  freq
0    Asian Restaurant  0.17
1                Bank  0.17
2   Electronics Store  0.17
3  Italian Restaurant  0.17
4            Pharmacy  0.17




#### Let's put that into a _pandas_ dataframe


First, let's write a function to sort the venues in descending order.


In [110]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.


In [111]:
num_top_venues = 6

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = humber_summit_grouped['Neighborhood']

for ind in np.arange(humber_summit_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(humber_summit_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue
0,Humber Summit (21),Asian Restaurant,Bank,Electronics Store,Italian Restaurant,Pharmacy,Shopping Mall


<a id='item4'></a>


## 4. Cluster Neighborhoods


Run _k_-means to cluster the neighborhood into 5 clusters.


In [112]:
# set number of clusters
kclusters = 1

humber_summit_grouped_clustering = humber_summit_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(humber_summit_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.


In [113]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

humber_summit_merged = humber_summit_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
humber_summit_merged = humber_summit_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

humber_summit_merged.head() # check the last columns!

,Hood,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue
0,Humber Summit,Humber Summit (21),43.748808,-79.557618,0,Asian Restaurant,Bank,Electronics Store,Italian Restaurant,Pharmacy,Shopping Mall
1,Humber Summit,Humber Summit (21),43.748808,-79.557618,0,Asian Restaurant,Bank,Electronics Store,Italian Restaurant,Pharmacy,Shopping Mall


Finally, let's visualize the resulting clusters


In [114]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(humber_summit_merged['Latitude'], humber_summit_merged['Longitude'], humber_summit_merged['Neighborhood'], humber_summit_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>


## 5. Examine Clusters


Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.


#### Cluster 1


In [115]:
humber_summit_merged.loc[humber_summit_merged['Cluster Labels'] == 0, humber_summit_merged.columns[[1] + list(range(5, humber_summit_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue
0,Humber Summit (21),Asian Restaurant,Bank,Electronics Store,Italian Restaurant,Pharmacy,Shopping Mall
1,Humber Summit (21),Asian Restaurant,Bank,Electronics Store,Italian Restaurant,Pharmacy,Shopping Mall
